# Aprendizado Ensemble e Florestas aleatórias

Se agregarmos as predições de um grupo de preditores (como classificadores ou regressores), iremos obter melhores predições do que com o melhor preditor individual. Um grupo de preditores é chamado de *ensemble* (agrupamento). Logo, essa técnica se chama *aprendizado emsemble* *, e um algoritmo de aprendizado ensemble é chamado de *método ensemble*.

Por exemplo, pode-se treinar um grupo de classificadores de uma árvore de decisão, cada um com um subconjunto aleatório diferente do conjunto de treinamento. Para efetuar as predições, obtem-se as predições de todas as árvores individuais e, em seguida, prediz a classe que obtém mais votos. Esse agrupamento de árvores de decisão se chama *Floresta aleatória*.

Usa-se com frequência os métodos ensemble no fim de um projeto, para combinar preditores bons que já tenham sido criados e criar um ainda melhor.

<small>* O termo aprendizado ensemble também é conhecido como aprendizagem por agrupamento ou por agregação.</small>

## Classificadores de votação

Pode-se ter um classificador de regressão logística, um classificador SVM, um classificador de floresta aleatória, um classificador k-enésimo vizinho mais próximo, e talvez mais alguns, cada um com uma acurácia de 80%. Uma forma simples de criar um classificador ainda melhor é agrupar as predições de cada classificador e predizer a classe que obtém mais votos. Esse classificador de predições de votos majoritários se chama classificador *hard voting*.

Ainda que cada classificador seja um *weak learner* (aprendiz fraco), o que significa que é ligeiramente melhor que uma suposição ao acaso, o agrupamento ainda pode ser um *strong learn* (aprendiz forte) e atingir uma acurácia alta, desde que exista um número suficiente de aprendizes fracos e eles sejam bastante diversificados.

Os métodos ensemble funcionam melhor quando os preditores são tão independentes dos outros quanto possivel. Uma forma de obter classificadores diversos é treiná-los usando algoritmos bastante diferentes. Isso aumenta a chance de eles cometerem erros com tipos muito diferentes, não correlacionados, melhorando a acurácia do agrupamento.

### Criando um classificador de votação com Scikit-Learn

Criando um classificador de votação constituído por 3 classificadores diversos.

In [10]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [21]:
from sklearn import datasets
iris = datasets.load_iris()
iris_df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

iris_df['target'] = (iris_df['target'] == 2).astype(np.int)

iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris_df[
  ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
  ], iris_df['target'], test_size=0.25, random_state=101)

In [23]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

In [24]:
voting_clf = VotingClassifier(
  estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
  voting='hard'
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

Vamos analizar a acurácia de cada classificador no conjunto de testes:

In [25]:
from sklearn.metrics import accuracy_score

In [26]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))


LogisticRegression 1.0
RandomForestClassifier 0.9736842105263158
SVC 1.0
VotingClassifier 1.0
